<a href="https://colab.research.google.com/github/ObiAU/MDS-Code/blob/main/MDS_Thesis_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# We can update this as we go
non_relevant = ['AGE(KA)', 'AGE(MA)', 'CITATION', 'AR36(CCMSTP/G)',
'AR39(CCMSTP/G)', 'AR39(MOL/G)', 'AR40(AT/G)',
'AR40(CCMSTP/G)', 'AR40(MOL/G)', 'AR40(PPM)', 'AR40(PPB)',
'HE3(CCMSTP/G)', 'HE4(CCMSTP/G)', 'K40(PPM)',
'NE20(CCMSTP/G)', 'CA42(PPM)', 'CA43(PPM)', 'GE73(PPM)',
'GE74(PPM)', 'KR84(CCMSTP/G)', 'MO95(PPM)', 'MO97(PPM)',
'MO98(PPM)', 'MO100(PPM)', 'XE132(CCMSTP/G)', 'RA226_ACT(BQ/KG)',
'ND143_ND144','EPSILON_ND','EPSILON_ND_INI','SM147_ND144',
'SR87_SR86', 'RB87_SR86', 'PB206_PB204','PB207_PB204',
'PB207_PB206', 'PB208_PB204', 'PB208_PB206','HF176_HF177', 'LU176_HF177',
'HE4_HE3', 'K40_AR36','K40_CA44', 'AR36_AR38', 'AR36_AR39',
'AR36_AR40','AR37_AR39', 'AR37_AR40','AR38_AR36', 'AR38_AR39', 'AR38_AR40',
'AR39_AR36','AR39_AR40', 'AR40_AR36', 'AR40_AR38', 'AR40_AR39', 'AR40_K40', 'PB207_U235',
'PB210_TH230_ACT', 'RA226_TH230_ACT', 'TH230_TH232_ACT', 'TH230_U238_ACT', 'TH232_PB204',
'TH232_PB208', 'TH232_U238', 'U234_U238_ACT','U235_PB204', 'U238_PB204', 'U238_PB206',
'U238_TH230_ACT', 'U238_TH232_ACT', 'N2(MOL/G)', 'D7LI(VS LSVEC)', 'D7LI(VS NBS8545)',
'D7LI(VS IRMM-016)', 'D11B(VS NBS951)', 'B11_B10', 'D13C(VS VPDB)', 'D15N(PER MIL)',
'D18O(PER MIL)', 'D18O(VS SMOW)', 'D18O(VS VSMOW)', 'D25MG(VS DSM3)', 'D26MG(VS DSM3)',
'CA40_CA44', 'D41K(VS SRM3141A)', 'D44_40CA(VS SRM915A)', 'D44_42CA(VS SRM915A)',
'D49TI(VS OL-TI)', 'D56FE(VS IRMM-014)', 'D57FE(VS IRMM-014)', 'D60NI(VS NBS986)',
'D66ZN(VS JMC3-0749)', 'D68ZN(VS JMC3-0749)', 'D98_95MO(PER MIL)', 'D98MO(VS NIST3134)',
'D98_95MO(VS NIST3134)', 'D137_134BA(VS SRM3104A)', 'D138_134BA(VS SRM3104A)',
'DD(PER MIL)', 'DD(VS SMOW)', 'DD(VS VSMOW)', 'SM2O3(WT%)', 'EU2O3(WT%)',
'GD2O3(WT%)','DY2O3(WT%)','SO2(WT%)','SO3(WT%)','S(WT%)','H2S(WT%)','LOI(WT%)',
'O(WT%)', 'YB2O3(WT%)', 'PR2O3(WT%)', 'AS2O3(WT%)', 'CH4(WT%)','O2(WT%)',
'CO(WT%)', 'H2O(WT%)', 'H2OP(WT%)', 'H2OM(WT%)','CO2(WT%)', 'OH(WT%)',
'HE3(CCMSTP/G)', 'HE4(CCMSTP/G)', 'NE20(CCMSTP/G)', 'AR36(CCMSTP/G)',
'AR39(CCMSTP/G)', 'AR39(MOL/G)', 'AR40(AT/G)', 'AR40(CCMSTP/G)', 'AR40(MOL/G)',
'AR40(PPM)', 'AR40(PPB)', 'K40(PPM)', 'CA42(PPM)', 'CA43(PPM)', 'GE73(PPM)',
'GE74(PPM)', 'KR84(CCMSTP/G)', 'MO95(PPM)', 'MO97(PPM)', 'MO98(PPM)',
'MO100(PPM)', 'XE132(CCMSTP/G)', 'ALBITE(MOL%)', 'ANORTHITE(MOL%)', 'ANNITE(MOL%)',
'ENSTATITE(MOL%)', 'FERROSILITE(MOL%)', 'MARGARITE(MOL%)','MUSCOVITE(MOL%)',
'ORTHOCLASE(MOL%)', 'PARAGONITE(MOL%)','PHLOGOPITE(MOL%)', 'WOLLASTONITE(MOL%)',
'N(PPM)', 'S(PPM)', 'BR(PPM)', 'GRAIN SIZE', 'SAMPLE NAME']

# You can scrap AJ-AN inclusive, AU, BR, CB-CI inclusive and CP.
# Other elements that won’t be of interest for your regression include BS-BV, CA

In [ ]:
ini = pd.read_excel('FULL_GEOROC_2022_12_SGFTFN_MICA.xlsx')
# Drop Non-Relevant Columns
df = ini.drop(non_relevant, axis = 1)
df.columns

####################################################################################

# INITIAL EDA

In [ ]:
iMajor = ['SIO2(WT%)', 'TIO2(WT%)', 'AL2O3(WT%)', 'FEO(WT%)',
         'MNO(WT%)', 'MGO(WT%)', 'CAO(WT%)', 'NA2O(WT%)',
         'K2O(WT%)', 'P2O5(WT%)']

iTrace = [
 'F(PPM)','CL(PPM)','SC(PPM)','TI(PPM)','V(PPM)',
 'CR(PPM)', 'CO(PPM)', 'NI(PPM)', 'CU(PPM)', 'ZN(PPM)', 'GA(PPM)',
'RB(PPM)', 'SR(PPM)',
 'Y(PPM)', 'ZR(PPM)', 'NB(PPM)', 'MO(PPM)', 'SN(PPM)',
  'CS(PPM)', 'BA(PPM)',
 'LA(PPM)', 'CE(PPM)', 'PR(PPM)', 'ND(PPM)', 'SM(PPM)', 'EU(PPM)',
 'GD(PPM)', 'TB(PPM)', 'DY(PPM)', 'HO(PPM)', 'ER(PPM)', 'TM(PPM)',
 'YB(PPM)', 'LU(PPM)', 'HF(PPM)', 'TA(PPM)', 'W(PPM)',
         'PB(PPM)', 'TH(PPM)', 'U(PPM)', 'N(PPM)', 'S(PPM)', 'BR(PPM)',]

iCategorical = ['TECTONIC SETTING', 'LOCATION',
'LAND/SEA (SAMPLING)', 'ROCK NAME', 'MINERAL',
'PRIMARY/SECONDARY', 'CRYSTAL', 'RIM/CORE (MINERAL GRAINS)','ROCK TEXTURE','ALTERATION',
'LOCATION COMMENT','SPOT']

In [ ]:
# Drop all NaN Li rows
ini = ini.dropna(subset=['LI(PPM)'])

## Missingness Proportion

In [ ]:
# # Calculate the percentage of missing values
# missing_major = ini[iMajor].isnull().mean() * 100

# # Plot
# plt.figure(figsize=(10, 5))
# missing_major.sort_values(ascending=True).plot(kind='barh')
# plt.title('Percentage of Missing Values in Major Element Subset')
# plt.xlabel('Percentage')
# plt.ylabel('Features')

# plt.savefig('missing_values_major.png')
# plt.show()

In [ ]:
# # Calculate the percentage of missing values
# missing_trace = ini[iTrace].isnull().mean() * 100

# # Plot
# plt.figure(figsize=(15, 10))
# missing_trace.sort_values(ascending=True).plot(kind='barh')
# plt.title('Percentage of Missing Values in Trace Element Subset')
# plt.xlabel('Percentage')
# plt.ylabel('Features')

# # Save the figure
# plt.savefig('missing_values_trace.png')

# plt.show()

In [ ]:
# # Calculate the percentage of missing values
# missing_categorical = ini[iCategorical].isnull().mean() * 100

# # Plot
# plt.figure(figsize=(10, 5))
# missing_categorical.sort_values(ascending=True).plot(kind='barh')
# plt.title('Percentage of Missing Values in Categorical Feature Subset')
# plt.xlabel('Percentage')
# plt.ylabel('Features')

# plt.tight_layout()
# plt.savefig('missing_values_categorical.png')
# plt.show()


## Missingness Patterns

In [ ]:
# # iMajor Element Subset
# plt.figure(figsize=(10, 8))  # Adjust these dimensions as needed
# msno.matrix(ini[iMajor])
# plt.title('Missingness Heatmap for Major Element Subset')
# plt.savefig('missing_heatmap_iMajor.png', bbox_inches='tight')
# plt.show()

# # msno.dendrogram(ini[iMajor])
# # plt.title('Missingness Dendrogram for Major Element Subset')
# # plt.tight_layout()
# # plt.savefig('missing_dendrogram_iMajor.png')
# # plt.show()

In [ ]:
# # iTrace Element Subset
# msno.matrix(ini[iTrace])
# plt.title('Missingness Heatmap for Trace Element Subset')
# plt.tight_layout()
# plt.savefig('missing_heatmap_iTrace.png')
# plt.show()

# msno.dendrogram(ini[iTrace])
# plt.title('Missingness Dendrogram for Trace Element Subset')
# plt.tight_layout()
# plt.savefig('missing_dendrogram_iTrace.png')
# plt.show()

In [ ]:
# # iCategorical Feature Subset
# msno.matrix(ini[iCategorical])
# plt.title('Missingness Heatmap for Categorical Feature Subset')
# plt.tight_layout()
# plt.savefig('missing_heatmap_iCategorical.png', bbox_inches='tight')
# plt.show()

# # msno.dendrogram(ini[iCategorical])
# # plt.title('Missingness Dendrogram for Categorical Feature Subset')
# # plt.tight_layout()
# # plt.savefig('missing_dendrogram_iCategorical.png')
# # plt.show()

#########################################################################

In [ ]:
excluded = ['CELADONITE', 'GLAUCONITE', 'HYDROMICA', 'HYDROMUSCOVITE',
            'MARGARITE', 'SERICITE', 'YANGZHUMINGITE']
extract = ~df['MINERAL'].isin(excluded)
df = df[extract]

In [ ]:
# We will write mol, wt%, and ppm calculation functions here
def wt2ppm(item): # to ppm
  return item * 10000

def ppm2wt(item): # to WT%
  return item / 10000

def ppb2m(item): # to ppm
  return item / 1000

def ppb2wt(item): # to WT%
  return item / 10000000

# Functions to convert materials from their oxide form to elements
def ZRO2(item):
  return item*(91.2/(91.2 + 32))

def HFO2(item):
  return item*(178.5/(178.5 + 32))

def THO2(item):
  return item*(232/(232 + 32))

def UO2(item):
  return item*(238/(238 + 32))

def UO3(item):
  return item*(238/(238 + 48))

def CR2O3(item):
  return item*((2*52)/(2*52 + 48))

def LA2O3(item):
  return item*((2*138.9)/(2*138.9 + 48))

def CE2O3(item):
  return item*((2*140.1)/(2*140.1 + 48))

def ND2O3(item):
  return item*((2*144.2)/(2*144.2 + 48))

def Y2O3(item):
  return item*((2*88.9)/(2*88.9 + 48))

def V2O3(item):
  return item*((2*50.9)/(2*50.9 + 48))

def V2O5(item):
  return item*((2*50.9)/(2*50.9 + 80))

def NB2O3(item):
  return item*((2*92.9)/(2*92.9 + 48))

def NB2O5(item):
  return item*((2*92.9)/(2*92.9 + 80))

def TA2O5(item):
  return item*((2*180.9)/(2*180.9 + 80))

def WO3(item):
  return item*((183.8)/(183.8 + 48))

def BAO(item):
  return item*(137.3/(137.3 + 16))

def SRO(item):
  return item*(87.6/(87.6 + 16))

def PBO(item):
  return item*(207.2/(207.2 + 16))

def SNO2(item):
  return item*(118.7/(118.7 + 32))

def NIO(item):
  return item*(58.7/(58.7 + 16))

def ZNO(item):
  return item*(65.4/(65.4 + 16))

def COO(item):
  return item*(58.9/(58.9 + 16))

def CUO(item):
  return item*(63.5/(63.5 + 16))

def CS2O(item):
  return item*((2*132.9)/(2*132.9 + 16))

def RB2O(item):
  return item*((2*85.5)/(2*85.5 + 16))

def LI2O(item):
  return item*((2*6.9)/(2*6.9 + 16))

def F2O(item):
  return item*((2*19)/(2*19 + 16))

def CL2O(item): # BZ
  return item*((2*35.5)/(2*35.5 + 16))


##############################################################################

# You can scrap AJ-AN inclusive, AU, BR, CA-CI inclusive and CP.
# Other elements that won’t be of interest for your regression include BS-BV
# Functions to convert certain elements to their oxide form
# Elements that should be left as oxide include SiO2, TiO2,
# Al2O3, FeO (or Fe2O3), MnO, MgO, CaO, Na2O, K2O, P2O5, H2O (if present).

# SiO2
def Si(revert):
  return revert*((28.1 + 32)/28.1)

# Al2O3
def Al(revert):
  return revert*((2*27 + 48)/(2*27))

# Fe2O3 to FeO
def FeO(revert):
  return revert*((2*(55.8 + 16))/(2*55.8 + 48))

# Fe to FeO
def Fe(revert):
  return revert*((55.8 + 16)/(58.8))

# MgO
def Mg(revert):
  return revert*((24.3 + 16)/24.3)

# Na2O
def Na(revert):
  return revert*((2*23 + 16)/(2*23))

# K2O
def K(revert):
  return revert*((2*39.1 + 16)/(2*39.1))

# P2O5
def P(revert):
  return revert*((2*31 + 80)/(2*31))

# CaO
def Ca(revert):
  return revert*((40.1 + 16)/40.1)

# MnO
def Mn(revert):
  return revert*((54.9 + 16)/54.9)

def convert_mixed_value(value):
    if isinstance(value, (float, int)):
        return float(value)
    if "\\" in value:
        parts = value.split('\\')
        number = parts[0].strip()
        return float(number)
    else:
        return float(value)


In [ ]:
# Merge FeOT and FeO
df['FEO(WT%)'] = df['FEOT(WT%)'].fillna(df['FEO(WT%)'])
# Merge Fe2O3T and Fe2O3
df['FE2O3(WT%)'] = df['FE2O3T(WT%)'].fillna(df['FE2O3(WT%)'])

In [ ]:
_numericals = ['SIO2(WT%)', 'TIO2(WT%)', 'AL2O3(WT%)', 'FEO(WT%)', 'CAO(WT%)',
    'MGO(WT%)', 'MNO(WT%)', 'K2O(WT%)', 'NA2O(WT%)', 'P2O5(WT%)', 'LI(PPM)',
    'BE(PPM)', 'B(PPM)', 'F(PPM)', 'CL(PPM)', 'SC(PPM)',
    'TI(PPM)', 'V(PPM)', 'CR(PPM)', 'CO(PPM)', 'NI(PPM)', 'CU(PPM)',
    'ZN(PPM)', 'GA(PPM)', 'GE(PPM)', 'AS(PPM)', 'SE(PPM)',
    'RB(PPM)', 'SR(PPM)', 'Y(PPM)', 'ZR(PPM)', 'NB(PPM)', 'MO(PPM)',
    'AG(PPM)', 'CD(PPM)', 'IN(PPM)', 'SN(PPM)', 'SB(PPM)', 'TE(PPM)',
    'CS(PPM)', 'BA(PPM)', 'LA(PPM)', 'CE(PPM)', 'PR(PPM)', 'ND(PPM)',
    'SM(PPM)', 'EU(PPM)', 'GD(PPM)', 'TB(PPM)', 'DY(PPM)', 'HO(PPM)',
    'ER(PPM)', 'TM(PPM)', 'YB(PPM)', 'LU(PPM)', 'HF(PPM)', 'TA(PPM)',
    'W(PPM)', 'AU(PPM)', 'HG(PPM)', 'TL(PPM)', 'PB(PPM)', 'BI(PPM)',
    'TH(PPM)', 'U(PPM)', 'CR2O3(WT%)', 'BAO(WT%)', 'NIO(WT%)',
            'LI2O(WT%)', 'RB2O(WT%)', 'K(WT%)']
for col in _numericals:
  df[col] = df[col].apply(convert_mixed_value)

In [ ]:
# Here, we convert each relevant column and will merge them in later code cells

# We use .pipe() to chain functions
# Converting WT% to WT% Oxides
# SiO2
df['SI(WT%)'] = df['SI(WT%)'].apply(Si)
df['SI(PPM)'] = df['SI(PPM)'].pipe(ppm2wt).pipe(Si)

# Al2O3
df['AL(WT%)'] = df['AL(WT%)'].apply(Al)
df['AL(PPM)'] = df['AL(PPM)'].pipe(ppm2wt).pipe(Al)

# Fe2O3 (FeOT and Fe2O3T are the only ones measured with the T method)
df['FE2O3(WT%)'] = df['FE2O3(WT%)'].apply(FeO)
# Fe
df['FE(WT%)'] = df['FE(WT%)'].apply(Fe)
df['FE(PPM)'] = df['FE(PPM)'].pipe(ppm2wt).pipe(Fe)

# MnO
df['MN(PPM)'] = df['MN(PPM)'].pipe(ppm2wt).pipe(Mn)

# MgO
df['MG(WT%)'] = df['MG(WT%)'].apply(Mg)
df['MG(PPM)'] = df['MG(PPM)'].pipe(ppm2wt).pipe(Mg)

# CaO
df['CA(PPM)'] = df['CA(PPM)'].pipe(ppm2wt).pipe(Ca)

# Na2O
df['NA(WT%)'] = df['NA(WT%)'].apply(Na)
df['NA(PPM)'] = df['NA(PPM)'].pipe(ppm2wt).pipe(Na)

# K2O
df['K(WT%)'] = df['K(WT%)'].apply(K)
df['K(PPM)'] = df['K(PPM)'].pipe(ppm2wt).pipe(K)

# P2O5
df['P(PPM)'] = df['P(PPM)'].pipe(ppm2wt).pipe(P)

# Trace elements; those in lower concentrations, should be left as ppm
# MAJOR ELEMENTS - SiO2, TiO2, Al2O3, FeO, MnO, MgO, CaO, Na2O, K2O, P2O5

# ZrO2 // ZR(PPM) - EP
df['ZRO2(WT%)'] = df['ZRO2(WT%)'].pipe(ZRO2).pipe(wt2ppm)

# HFO2 // HF(PPM) - FT
df['HFO2(WT%)'] = df['HFO2(WT%)'].pipe(HFO2).pipe(wt2ppm)

# THO2 // TH(PPM) - GD
df['THO2(WT%)'] = df['THO2(WT%)'].pipe(THO2).pipe(wt2ppm)

# UO2, UO3 // U(PPM), U(PPB) - GE, GF
df['UO2(WT%)'] = df['UO2(WT%)'].pipe(UO2).pipe(wt2ppm)
df['UO3(WT%)'] = df['UO3(WT%)'].pipe(UO3).pipe(wt2ppm)
df['U(PPB)'] = df['U(PPB)'].pipe(ppb2m)

# CR2O3 // CR(PPM) - DX
df['CR2O3(WT%)'] = df['CR2O3(WT%)'].pipe(CR2O3).pipe(wt2ppm)
df['CR(WT%)'] = df['CR(WT%)'].pipe(wt2ppm)

# LA2O3 // LA(PPM) - FF
df['LA2O3(WT%)'] = df['LA2O3(WT%)'].pipe(LA2O3).pipe(wt2ppm)

# CE2O3 // CE(PPM) - FG
df['CE2O3(WT%)'] = df['CE2O3(WT%)'].pipe(CE2O3).pipe(wt2ppm)

# ND2O3 // ND(PPM) - FI
df['ND2O3(WT%)'] = df['ND2O3(WT%)'].pipe(ND2O3).pipe(wt2ppm)

# Y2O3 // Y(PPM) - EO
df['Y2O3(WT%)'] = df['Y2O3(WT%)'].pipe(Y2O3).pipe(wt2ppm)

# V2O3, V2O5 // V(PPM) - DW
df['V2O3(WT%)'] = df['V2O3(WT%)'].pipe(V2O3).pipe(wt2ppm)
df['V2O5(WT%)'] = df['V2O3(WT%)'].pipe(V2O5).pipe(wt2ppm)

# NB2O3, NB2O5 // NB(PPM) - EQ
df['NB2O3(WT%)'] = df['NB2O3(WT%)'].pipe(NB2O3).pipe(wt2ppm)
df['NB2O5(WT%)'] = df['NB2O3(WT%)'].pipe(NB2O5).pipe(wt2ppm)

# TA2O5 // TA(PPM) - FU
df['TA2O5(WT%)'] = df['TA2O5(WT%)'].pipe(TA2O5).pipe(wt2ppm)

# WO3 // W(PPM) - FV
df['WO3(WT%)'] = df['WO3(WT%)'].pipe(WO3).pipe(wt2ppm)

# BAO // BA(PPM) - FE
df['BAO(WT%)'] = df['BAO(WT%)'].pipe(BAO).pipe(wt2ppm)

# SRO // SR(PPM) - EN
df['SRO(WT%)'] = df['SRO(WT%)'].pipe(SRO).pipe(wt2ppm)

# PBO // PB(PPM) - FZ
df['PBO(WT%)'] = df['PBO(WT%)'].pipe(PBO).pipe(wt2ppm)
df['PB(PPB)'] = df['PB(PPB)'].pipe(ppb2m)

# SNO2 // SN(PPM) - EZ
df['SNO2(WT%)'] = df['SNO2(WT%)'].pipe(SNO2).pipe(wt2ppm)

# NIO // NI(PPM) - EB
df['NIO(WT%)'] = df['NIO(WT%)'].pipe(NIO).pipe(wt2ppm)

# ZNO // ZN(PPM) - ED
df['ZNO(WT%)'] = df['ZNO(WT%)'].pipe(ZNO).pipe(wt2ppm)

# COO // CO(PPM) - EA
df['COO(WT%)'] = df['COO(WT%)'].pipe(COO).pipe(wt2ppm)

# CUO // CU(PPM) - EC
df['CUO(WT%)'] = df['CUO(WT%)'].pipe(CUO).pipe(wt2ppm)

# CS2O // CS(PPM) - FD
df['CS2O(WT%)'] = df['CS2O(WT%)'].pipe(CS2O).pipe(wt2ppm)

# RB2O // RB(PPM) - EM
df['RB2O(WT%)'] = df['RB2O(WT%)'].pipe(RB2O).pipe(wt2ppm)

# LI2O // LI(PPM) - CT
df['LI2O(WT%)'] = df['LI2O(WT%)'].pipe(LI2O).pipe(wt2ppm)

# F2O // F(PPM) - CX
df['F(WT%)'] = df['F(WT%)'].pipe(wt2ppm)
df['F2O(WT%)'] = df['F2O(WT%)'].pipe(F2O).pipe(wt2ppm)

# CL2O // CL(PPM) - DF
df['CL(WT%)'] = df['CL(WT%)'].pipe(wt2ppm)
df['CL2O(WT%)'] = df['CL2O(WT%)'].pipe(CL2O).pipe(wt2ppm)

In [ ]:
# Oxides (We will leave major elements in WT% form)

# SIO2
df['SIO2(WT%)'] = df['SIO2(WT%)'].fillna(df['SI(PPM)']).fillna(df['SI(WT%)'])

# AL2O3
df['AL2O3(WT%)'] = df['AL2O3(WT%)'].fillna(df['AL(WT%)']).fillna(df['AL(PPM)'])

# FEO
df['FEO(WT%)'] = df['FEO(WT%)'].fillna(df['FE2O3(WT%)']).fillna(df['FE(WT%)']).fillna(df['FE(PPM)'])

# MNO
df['MNO(WT%)'] = df['MNO(WT%)'].fillna(df['MN(PPM)'])

# MGO
df['MGO(WT%)'] = df['MGO(WT%)'].fillna(df['MG(PPM)']).fillna(df['MG(WT%)'])

# CAO
df['CAO(WT%)'] = df['CAO(WT%)'].fillna(df['CA(PPM)'])

# NA2O
df['NA2O(WT%)'] = df['NA2O(WT%)'].fillna(df['NA(WT%)']).fillna(df['NA(PPM)'])

# K20
df['K2O(WT%)'] = df['K2O(WT%)'].fillna(df['K(WT%)']).fillna(df['K(PPM)'])

# P2O5
df['P2O5(WT%)'] = df['P2O5(WT%)'].fillna(df['P(PPM)'])

# Trace elements should be left in ppm format

# ZR
df['ZR(PPM)'] = df['ZR(PPM)'].fillna(df['ZRO2(WT%)'])

# HF
df['HF(PPM)'] = df['HF(PPM)'].fillna(df['HFO2(WT%)'])

# TH
df['TH(PPM)'] = df['TH(PPM)'].fillna(df['THO2(WT%)'])

# U
df['U(PPM)'] = df['U(PPM)'].fillna(df['UO2(WT%)']).fillna(df['UO3(WT%)']).fillna(df['U(PPB)'])

# CR
df['CR(PPM)'] = df['CR(PPM)'].fillna(df['CR2O3(WT%)']).fillna(df['CR(WT%)'])

# LA
df['LA(PPM)'] = df['LA(PPM)'].fillna(df['LA2O3(WT%)'])

# CE
df['CE(PPM)'] = df['CE(PPM)'].fillna(df['CE2O3(WT%)'])

# ND
df['ND(PPM)'] = df['ND(PPM)'].fillna(df['ND2O3(WT%)'])

# Y
df['Y(PPM)'] = df['Y(PPM)'].fillna(df['Y2O3(WT%)'])

# V
df['V(PPM)'] = df['V(PPM)'].fillna(df['V2O3(WT%)']).fillna(df['V2O5(WT%)'])

# NB
df['NB(PPM)'] = df['NB(PPM)'].fillna(df['NB2O3(WT%)']).fillna(df['NB2O5(WT%)'])

# TA
df['TA(PPM)'] = df['TA(PPM)'].fillna(df['TA2O5(WT%)'])

# W
df['W(PPM)'] = df['W(PPM)'].fillna(df['WO3(WT%)'])

# BA
df['BA(PPM)'] = df['BA(PPM)'].fillna(df['BAO(WT%)'])

# SR
df['SR(PPM)'] = df['SR(PPM)'].fillna(df['SRO(WT%)'])

# PB
df['PB(PPM)'] = df['PB(PPM)'].fillna(df['PBO(WT%)']).fillna(df['PB(PPB)'])

# SN
df['SN(PPM)'] = df['SN(PPM)'].fillna(df['SNO2(WT%)'])

# NI
df['NI(PPM)'] = df['NI(PPM)'].fillna(df['NIO(WT%)'])

# ZN
df['ZN(PPM)'] = df['ZN(PPM)'].fillna(df['ZNO(WT%)'])

# CO
df['CO(PPM)'] = df['CO(PPM)'].fillna(df['COO(WT%)'])

# CU
df['CU(PPM)'] = df['CU(PPM)'].fillna(df['CUO(WT%)'])

# CS
df['CS(PPM)'] = df['CS(PPM)'].fillna(df['CS2O(WT%)'])

# RB
df['RB(PPM)'] = df['RB(PPM)'].fillna(df['RB2O(WT%)'])

# LI
df['LI(PPM)'] = df['LI(PPM)'].fillna(df['LI2O(WT%)'])

# F
df['F(PPM)'] = df['F(PPM)'].fillna(df['F(WT%)']).fillna(df['F2O(WT%)'])

# CL
df['CL(PPM)'] = df['CL(PPM)'].fillna(df['CL(WT%)']).fillna(df['CL2O(WT%)'])


In [ ]:
postmerged = ['SI(WT%)', 'SI(PPM)', 'AL(WT%)', 'AL(PPM)', 'FE2O3(WT%)',
              'FE2O3T(WT%)', 'FEOT(WT%)', 'MN(PPM)', 'MG(WT%)', 'MG(PPM)',
              'CA(PPM)', 'NA(WT%)', 'NA(PPM)', 'K(WT%)', 'K(PPM)',
              'P(PPM)', 'ZRO2(WT%)', 'HFO2(WT%)', 'THO2(WT%)', 'UO2(WT%)',
              'UO3(WT%)', 'U(PPB)', 'CR2O3(WT%)', 'CR(WT%)', 'LA2O3(WT%)',
              'CE2O3(WT%)', 'ND2O3(WT%)', 'Y2O3(WT%)', 'V2O3(WT%)', 'V2O5(WT%)',
              'NB2O3(WT%)', 'NB2O5(WT%)', 'TA2O5(WT%)', 'WO3(WT%)', 'BAO(WT%)',
              'SRO(WT%)', 'PBO(WT%)', 'PB(PPB)','SNO2(WT%)', 'NIO(WT%)',
              'ZNO(WT%)', 'COO(WT%)', 'CUO(WT%)', 'CS2O(WT%)', 'RB2O(WT%)',
              'LI2O(WT%)', 'F(WT%)', 'F2O(WT%)', 'CL(WT%)', 'CL2O(WT%)',
              'FE(WT%)', 'FE(PPM)', 'AR(MOL/G)']

df = df.drop(postmerged, axis = 1)
df.columns

In [ ]:
df = df.dropna(subset=['LI(PPM)'])
df.reset_index(drop = True, inplace = True)

In [ ]:
# Categorize rock name
def categorize_rock(item):
  # if isinstance(item, float):
  #   return item
  if item == 'MONZOGRANITE':
    return 'SYENITE'
  elif item in ['RHYOLITE', 'RHYODACITE']:
    return 'GRANITE'
  elif 'GRANITE' in item:
    return 'GRANITE'
  elif item in ['APLITE', 'PEGMATITE', 'GREISEN']:
    return 'GREISEN'
  elif item in ['DACITE', 'TONALITE', 'TRONDHJEMITE']:
    return 'GRANODIORITE'
  elif 'GRANODIORITE' in item:
    return 'GRANODIORITE'
  elif 'DIORITE' in item:
    return 'DIORITE'
  elif item == 'ANDESITE':
    return 'DIORITE'
  elif 'GABBRO' in item:
    return 'GABBRO'
  elif 'SYENITE' in item:
    return 'SYENITE'
  elif item in ['MONZONITE', 'MELASYENITE', 'PHONOLITE', 'TRACHYTE']:
    return 'SYENITE'
  elif 'CARBONATITE' in item:
    return 'CARBONATITE'
  elif item in ['LAMPROPHYRE', 'VOGESITE', 'NEPHELINITE']:
    return 'CARBONATITE'
  elif 'PHLOGOPITE' in item:
    return 'NOT GIVEN'
  elif 'XENOLITH' in item:
    return 'XENOLITH'
  elif item in ['CLINOPYROXENITE', 'HARZBURGITE', 'WEHRLITE', 'ECLOGITE']:
    return 'XENOLITH'
  else:
    return item

df['ROCK NAME'] = df['ROCK NAME'].apply(categorize_rock)

In [ ]:
def simplify_location(df, col):
    df[col] = df[col].apply(lambda x: x.split('/')[0].strip())
    return df

df = simplify_location(df, 'LOCATION')

In [ ]:
numericals = ['SIO2(WT%)', 'TIO2(WT%)', 'AL2O3(WT%)', 'FEO(WT%)', 'CAO(WT%)',
       'MGO(WT%)', 'MNO(WT%)', 'K2O(WT%)', 'NA2O(WT%)', 'P2O5(WT%)', 'LI(PPM)',
       'BE(PPM)', 'B(PPM)', 'F(PPM)', 'CL(PPM)', 'SC(PPM)',
       'TI(PPM)', 'V(PPM)', 'CR(PPM)', 'CO(PPM)', 'NI(PPM)', 'CU(PPM)',
       'ZN(PPM)', 'GA(PPM)', 'GE(PPM)', 'AS(PPM)', 'SE(PPM)',
       'RB(PPM)', 'SR(PPM)', 'Y(PPM)', 'ZR(PPM)', 'NB(PPM)', 'MO(PPM)',
       'AG(PPM)', 'CD(PPM)', 'IN(PPM)', 'SN(PPM)', 'SB(PPM)', 'TE(PPM)',
       'CS(PPM)', 'BA(PPM)', 'LA(PPM)', 'CE(PPM)', 'PR(PPM)', 'ND(PPM)',
       'SM(PPM)', 'EU(PPM)', 'GD(PPM)', 'TB(PPM)', 'DY(PPM)', 'HO(PPM)',
       'ER(PPM)', 'TM(PPM)', 'YB(PPM)', 'LU(PPM)', 'HF(PPM)', 'TA(PPM)',
       'W(PPM)', 'AU(PPM)', 'HG(PPM)', 'TL(PPM)', 'PB(PPM)', 'BI(PPM)',
       'TH(PPM)', 'U(PPM)', 'LATITUDE (MIN.)', 'LATITUDE (MAX.)',
       'LONGITUDE (MIN.)', 'LONGITUDE (MAX.)', 'ELEVATION (MIN.)',
       'ELEVATION (MAX.)', 'DRILLING DEPTH (MIN.)', 'DRILLING DEPTH (MAX.)' ]

In [ ]:
from sklearn.impute import KNNImputer

# Instantiate the KNN imputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors if needed

# Perform KNN imputation on the selected numerical columns
df[numericals] = imputer.fit_transform(df[numericals])


####################################################################################
# EDA BEGINS HERE

### Ideas

Lithium Concentration is the dependant variable. It is our primary interest and so we should perform this EDA around that.

* Lithium against Major elements
* Lithium against trace elements

In [ ]:
Major1 = ['SIO2(WT%)', 'TIO2(WT%)', 'AL2O3(WT%)', 'FEO(WT%)',
         'MNO(WT%)', 'MGO(WT%)', 'CAO(WT%)', 'NA2O(WT%)',
         'K2O(WT%)', 'P2O5(WT%)', 'LI(PPM)']

Major = ['SIO2(WT%)', 'TIO2(WT%)', 'AL2O3(WT%)', 'FEO(WT%)',
         'MNO(WT%)', 'MGO(WT%)', 'CAO(WT%)', 'NA2O(WT%)',
         'K2O(WT%)', 'P2O5(WT%)']

Trace1 = ['LI(PPM)',
 'F(PPM)','CL(PPM)','SC(PPM)','TI(PPM)','V(PPM)',
 'CR(PPM)', 'CO(PPM)', 'NI(PPM)', 'CU(PPM)', 'ZN(PPM)', 'GA(PPM)',
'RB(PPM)', 'SR(PPM)',
 'Y(PPM)', 'ZR(PPM)', 'NB(PPM)', 'MO(PPM)', 'SN(PPM)',
  'CS(PPM)', 'BA(PPM)',
 'LA(PPM)', 'CE(PPM)', 'PR(PPM)', 'ND(PPM)', 'SM(PPM)', 'EU(PPM)',
 'GD(PPM)', 'TB(PPM)', 'DY(PPM)', 'HO(PPM)', 'ER(PPM)', 'TM(PPM)',
 'YB(PPM)', 'LU(PPM)', 'HF(PPM)', 'TA(PPM)', 'W(PPM)',
         'PB(PPM)', 'TH(PPM)', 'U(PPM)']

Trace = ['F(PPM)','CL(PPM)','SC(PPM)','TI(PPM)','V(PPM)',
 'CR(PPM)', 'CO(PPM)', 'NI(PPM)', 'CU(PPM)', 'ZN(PPM)', 'GA(PPM)',
'RB(PPM)', 'SR(PPM)',
 'Y(PPM)', 'ZR(PPM)', 'NB(PPM)', 'MO(PPM)', 'SN(PPM)',
  'CS(PPM)', 'BA(PPM)',
 'LA(PPM)', 'CE(PPM)', 'PR(PPM)', 'ND(PPM)', 'SM(PPM)', 'EU(PPM)',
 'GD(PPM)', 'TB(PPM)', 'DY(PPM)', 'HO(PPM)', 'ER(PPM)', 'TM(PPM)',
 'YB(PPM)', 'LU(PPM)', 'HF(PPM)', 'TA(PPM)', 'W(PPM)',
         'PB(PPM)', 'TH(PPM)', 'U(PPM)']

## MAJOR

In [ ]:
# Major elements distribution examination
sns.set(style='whitegrid', palette='Set1')
plt.figure(figsize=(15, 6))
for i, feature in enumerate(Major):
    plt.subplot(2, 6, i+1)
    sns.histplot(df[feature], kde=True)
    plt.title(feature)
plt.tight_layout()
plt.savefig('MajorDense.png')
plt.show()

In [ ]:
# Plot correlation heatmap for 'Major' category
plt.figure(figsize=(10, 8))
corr_major = df[Major1].corr()
sns.heatmap(corr_major, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap - Major Elements")
plt.tight_layout()
plt.savefig('MajorCorr.png')
plt.show()

In [ ]:
# Plot pair plots for selected features in 'Major' category
sns.pairplot(df[Major], diag_kind='kde')
# plt.suptitle("Pair Plots - Major Elements", y=1.02)
plt.title('Pair Plot - Trace Elements')
plt.tight_layout()
plt.savefig('MajorPair.png')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame and you want to predict 'LI(PPM)'
X = df[Major1].drop('LI(PPM)', axis=1)  # input features
y = df['LI(PPM)']  # target variable

# Create a random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the regressor
rf.fit(X, y)

# Get feature importances
importances = rf.feature_importances_

# Sort the features by importance
sorted_indices = np.argsort(importances)[::-1]

# Create a DataFrame for the sorted feature importances
feature_ranking_df = pd.DataFrame({
    'Feature': X.columns[sorted_indices],
    'Importance': importances[sorted_indices]
})

# Plotting the horizontal bar chart for feature importances
plt.figure(figsize=(10, 10))  # You can adjust the size as per your needs
plt.barh(feature_ranking_df['Feature'], feature_ranking_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance - Major Elements')
plt.gca().invert_yaxis()  # To display the most important features at the top

# Display importance values at the end of each bar
for index, value in enumerate(feature_ranking_df['Importance']):
    plt.text(value, index, f'{value:.3f}')  # Displaying values rounded to 3 decimal places

plt.tight_layout()
plt.savefig('MajorImportance.png')
plt.show()


In [ ]:
# Set the default color palette for seaborn
sns.set_palette(['red'])

# Select the top N features
N = 5  # for example
selected_features = [X.columns[i] for i in sorted_indices[:N]]

# Now you can create a pair plot with the selected features
sns.pairplot(df[selected_features], diag_kind='kde')
plt.suptitle('Pair Plot - Major Elements', y=1.02)
plt.tight_layout()
plt.savefig('MajorPair.png', bbox_inches='tight')
plt.show()

# Reset the palette if needed
sns.set_palette('deep')  # 'deep' is the default seaborn palette

## TRACE

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame and you want to predict 'LI(PPM)'
X = df[Trace1].drop('LI(PPM)', axis=1)  # input features
y = df['LI(PPM)']  # target variable

# Create a random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the regressor
rf.fit(X, y)

# Get feature importances
importances = rf.feature_importances_

# Sort the features by importance
sorted_indices = np.argsort(importances)[::-1]

# Create a DataFrame for the sorted feature importances
feature_ranking_df = pd.DataFrame({
    'Feature': X.columns[sorted_indices],
    'Importance': importances[sorted_indices]
})

# Plotting the horizontal bar chart for feature importances
plt.figure(figsize=(10, 10))  # You can adjust the size as per your needs
plt.barh(feature_ranking_df['Feature'], feature_ranking_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance - Trace Elements')
plt.gca().invert_yaxis()  # To display the most important features at the top

# Display importance values at the end of each bar
for index, value in enumerate(feature_ranking_df['Importance']):
    plt.text(value, index, f'{value:.3f}')  # Displaying values rounded to 3 decimal places

plt.tight_layout()
plt.savefig('TraceImportance.png')
plt.show()


In [ ]:
# Select the top N features
N_trace = 10
selected_features_trace = feature_ranking_df['Feature'][:N_trace].tolist()

N_trace5 = 5
selected_features_trace5 = feature_ranking_df['Feature'][:N_trace5].tolist()

In [ ]:
sns.pairplot(df[selected_features_trace5], diag_kind='kde')
plt.suptitle('Pair Plot - Trace Elements', y=1.02)
plt.tight_layout()
plt.savefig('TracePair.png', bbox_inches='tight')  # This should prevent cropping
plt.show()

In [ ]:
# Major elements distribution examination
sns.set(style='whitegrid', palette='Set1')
plt.figure(figsize=(15, 6))
for i, feature in enumerate(selected_features_trace):
    plt.subplot(2, 6, i+1)
    sns.histplot(df[feature], kde=True)
    plt.title(feature)
plt.tight_layout()
plt.savefig('TraceDense.png')
plt.show()

In [ ]:
# plt.figure(figsize=(15, 10))
# for i, feature in enumerate(selected_features_trace):
#     plt.subplot(N_trace, 1, i+1)
#     sns.histplot(df[feature], kde=True)
#     plt.title(feature)
# plt.tight_layout()
# plt.
# plt.show()

In [ ]:
# Add 'LI(PPM)' to the list of selected features
selected_features_trace_with_LI = selected_features_trace + ['LI(PPM)']

# Calculate correlation matrix for the selected features including 'LI(PPM)'
corr_trace_selected = df[selected_features_trace_with_LI].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_trace_selected, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap - Selected Trace Elements including LI(PPM)")
plt.tight_layout()
plt.savefig('TraceCorr.png')
plt.show()


## CATEGORICAL

In [ ]:
Categorical = ['TECTONIC SETTING', 'LOCATION',
'LAND/SEA (SAMPLING)', 'ROCK NAME', 'MINERAL',
'PRIMARY/SECONDARY']

#'CRYSTAL', 'RIM/CORE (MINERAL GRAINS)','ROCK TEXTURE','ALTERATION',
#'LOCATION COMMENT','SPOT',

* Crystal and Rim/Core have too many NAN values

In [ ]:
df[Categorical].isnull().sum()

In [ ]:
# import scipy.stats as stats

# def eta_squared(categorical, continuous):
#     categories = categorical.unique()
#     N = len(continuous)
#     ss_total = np.sum((continuous - np.mean(continuous))**2)

#     sum_sq_bw = sum([len(continuous[categorical == cat]) * (np.mean(continuous[categorical == cat]) - np.mean(continuous))**2
#                      for cat in categories])
#     return sum_sq_bw / ss_total

# # Compute the eta squared statistic for each categorical variable with respect to LI(PPM)
# eta2_values = [eta_squared(df[cat], df['LI(PPM)']) for cat in Categorical]

# # Display the results
# for cat, value in zip(Categorical, eta2_values):
#     print(f"{cat}: {value:.4f}")
# plt.figure(figsize=(10, 5))
# sns.barplot(Categorical, eta2_values, palette="coolwarm")
# plt.xticks(rotation=45)
# plt.ylabel("$\eta^2$ value")
# plt.title("$\eta^2$ values for Categorical Variables with respect to LI(PPM)")
# plt.tight_layout()
# plt.show()


In [ ]:
# Compute the Cramer's V statistic for each pair of categorical variables
v_matrix = np.zeros((len(Categorical), len(Categorical)))
for i, col1 in enumerate(Categorical):
    for j, col2 in enumerate(Categorical):
        if i == j:
            continue
        confusion_matrix = pd.crosstab(df[col1], df[col2])
        chi2, p, dof, expected = chi2_contingency(confusion_matrix)
        n = confusion_matrix.sum().sum()
        phi = np.sqrt(chi2/n)
        k = min(confusion_matrix.shape[0], confusion_matrix.shape[1])
        v = np.sqrt(phi/(k-1))
        v_matrix[i, j] = v

# Create a heatmap to visualize the Cramer's V matrix
plt.figure(figsize=(10, 8))
sns.heatmap(v_matrix, xticklabels=Categorical, yticklabels=Categorical, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Cramer's V Heatmap for Categorical Variables")
plt.tight_layout()
plt.savefig('CateHeat')
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='TECTONIC SETTING', palette='Set2')
plt.xticks(rotation=90)
plt.title('Count Plot - Tectonic Setting')
plt.tight_layout()
plt.savefig('TectonicCount.png')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='LOCATION', palette='Set2')
plt.xticks(rotation=90)
plt.title('Count Plot - Location')
plt.tight_layout()
plt.savefig('LocationCount.png')
plt.show()

In [ ]:
plt.figure(figsize=(7.5, 6))
sns.countplot(data=df, x='LAND/SEA (SAMPLING)', palette='Set2')
plt.xticks(rotation=90)
plt.title('Count Plot - Land/Sea (Sampling)')
plt.tight_layout()
plt.savefig('LandSeaCount.png')
plt.show()

In [ ]:
# # !pip install squarify
# import squarify

# rock_name_counts = df['ROCK NAME'].value_counts()
# labels = rock_name_counts.index
# sizes = rock_name_counts.values

# plt.figure(figsize=(15, 10))
# squarify.plot(sizes=sizes, label=labels, alpha=0.6, color=sns.color_palette("Set2", len(sizes)))
# plt.title('Tree Map of ROCK NAME Categories')
# plt.axis('off')
# plt.show()


In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.countplot(data=df, x='MINERAL', palette='Set2')
plt.xticks(rotation=90)
plt.title('MINERAL')

plt.subplot(1, 2, 2)
sns.countplot(data=df, x='PRIMARY/SECONDARY', palette='Set2')
plt.xticks(rotation=90)
plt.title('PRIMARY/SECONDARY')

plt.tight_layout()
plt.savefig('PrimaryXMineralCount.png')
plt.show()


######################################################

# PRINCIPLE COMPONENT ANALYSIS

In [ ]:
Trace = ['F(PPM)','CL(PPM)','SC(PPM)','TI(PPM)','V(PPM)',
 'CR(PPM)', 'CO(PPM)', 'NI(PPM)', 'CU(PPM)', 'ZN(PPM)', 'GA(PPM)',
'RB(PPM)', 'SR(PPM)',
 'Y(PPM)', 'ZR(PPM)', 'NB(PPM)', 'MO(PPM)', 'SN(PPM)',
  'CS(PPM)', 'BA(PPM)',
 'LA(PPM)', 'CE(PPM)', 'PR(PPM)', 'ND(PPM)', 'SM(PPM)', 'EU(PPM)',
 'GD(PPM)', 'TB(PPM)', 'DY(PPM)', 'HO(PPM)', 'ER(PPM)', 'TM(PPM)',
 'YB(PPM)', 'LU(PPM)', 'HF(PPM)', 'TA(PPM)', 'W(PPM)',
         'PB(PPM)', 'TH(PPM)', 'U(PPM)']

Major = ['SIO2(WT%)', 'TIO2(WT%)', 'AL2O3(WT%)', 'FEO(WT%)',
         'MNO(WT%)', 'MGO(WT%)', 'CAO(WT%)', 'NA2O(WT%)',
         'K2O(WT%)', 'P2O5(WT%)']

In [ ]:
all_features = Major + Trace

# Extract the relevant columns from the DataFrame
df_subset = df[all_features]

In [ ]:
# 1. Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_subset)

# 2. Perform PCA
pca = PCA()
principal_components = pca.fit_transform(scaled_data)

# Convert PCA results to a DataFrame
pca_df = pd.DataFrame(data=principal_components)

# 3. Visualize explained variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()

plt.figure(figsize=(10, 5))

plt.bar(range(len(explained_variance)), explained_variance, alpha=0.5, label='Explained Variance by Component')
plt.plot(range(len(cumulative_variance)), cumulative_variance, marker='o', color='red', label='Cumulative Explained Variance')

plt.title('Explained Variance by Principal Component')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.legend()
plt.tight_layout()
plt.savefig('PCAScree.png')
plt.show()


In [ ]:
# Extracting components for 85% explained variance:
num_components = np.argmax(cumulative_variance > 0.85) + 1
print(f"The number of components needed to explain 85% of the variance is: {num_components}")

# Extracting and inspecting those components:
important_components_df = pca_df.iloc[:, :num_components]
print(important_components_df)

In [ ]:
# Split the dataset into training and testing sets
X = df_subset  # Your features
y = df['LI(PPM)']  # Your dependent variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Baseline model (You can choose another if desired)
model = LinearRegression()

performance_record = {}

# Test for each number of components
for num in range(1, len(explained_variance) + 1):
    # Applying PCA transformation
    pca = PCA(n_components=num)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Train and predict
    model.fit(X_train_pca, y_train)
    predictions = model.predict(X_test_pca)

    # Evaluate the performance (using Mean Squared Error here, you can choose another metric if desired)
    mse = mean_squared_error(y_test, predictions)

    performance_record[num] = mse

# Find the number of components with the best performance (lowest MSE)
best_num_components = min(performance_record, key=performance_record.get)
print(f"Best number of components is: {best_num_components} with MSE: {performance_record[best_num_components]}")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X = df_subset
y = df['LI(PPM)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Linear Regression with PCA
linear_reg_performance = {}
for num in range(1, len(explained_variance) + 1):
    pca = PCA(n_components=num)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_pca, y_train)
    predictions = model.predict(X_test_pca)

    mse = mean_squared_error(y_test, predictions)
    linear_reg_performance[num] = mse

best_linear_components = min(linear_reg_performance, key=linear_reg_performance.get)

# Polynomial Regression
degree = 2
poly_features = PolynomialFeatures(degree=degree, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

poly_reg_performance = {}
for n in range(1, 51):  # Assuming 50 components
    model = LinearRegression()
    model.fit(X_train_poly[:, :n], y_train)
    predictions = model.predict(X_test_poly[:, :n])
    mse = mean_squared_error(y_test, predictions)
    poly_reg_performance[n] = mse

best_poly_components = min(poly_reg_performance, key=poly_reg_performance.get)

# Display results in a table using pandas DataFrame
results_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Polynomial Regression'],
    'Best no. of components': [best_linear_components, best_poly_components],
    'MSE': [linear_reg_performance[best_linear_components], poly_reg_performance[best_poly_components]]
})

print(results_df)
